# Twin4Build: Parameter Estimation Example

This notebook demonstrates how to perform parameter estimation on a building model using Twin4Build. You'll learn how to:
1. Load a pre-built model from a semantic file
2. Set up simulation parameters and time periods
3. Define target parameters for estimation
4. Configure measuring devices for calibration
5. Run parameter estimation using automatic differentiation
6. Visualize and compare results before and after calibration

This example is particularly useful for understanding:
- How to calibrate building models with real data
- Parameter estimation techniques in building simulation
- Model validation and verification
- Automatic differentiation for gradient-based optimization
- Thermal and mass balance parameter calibration

## 1. Setup
First, let's install and import the necessary packages. If you're running this in Google Colab, uncomment the pip install line.
Note that colab has a dependency issue and asks you to restart the runtime. It will work after you restart the runtime and run the cell a 2nd time.  

In [1]:
# %pip install twin4build # Uncomment in google colab
import twin4build as tb
import datetime
from dateutil import tz
import twin4build.examples.utils as utils

## 2. Load the Model

We'll load a pre-built model from a semantic file. This model contains a building space with various components including:
- Building space with thermal and mass parameters
- Space heater for heating
- Temperature and CO2 controllers
- Valves and dampers for control
- Various sensors for measurement

In [2]:
# Create a new model
model = tb.Model(id="estimator_example")

# Load the model from semantic file
filename_simulation = utils.get_path(["estimator_example", "semantic_model.ttl"])
model.load(simulation_model_filename=filename_simulation, verbose=0)
print(model)

Drawing simulation model...[OK]


ValueError: Line not found: 'Loading model'current level: 0verbose: 3

## (Change filepaths to match this local machine)

This step is only necesarry if the (.ttl) file you are loading was not serialized on your machine. (In this case, we need to change tyhe machine-dependent file paths )

In [ ]:
model.components["020B_temperature_sensor"].filename = utils.get_path(["estimator_example", "temperature_sensor.csv"])
model.components["020B_co2_sensor"].filename = utils.get_path(["estimator_example", "co2_sensor.csv"])
model.components["020B_valve_position_sensor"].filename = utils.get_path(["estimator_example", "valve_position_sensor.csv"])
model.components["020B_damper_position_sensor"].filename = utils.get_path(["estimator_example", "damper_position_sensor.csv"])
model.components["BTA004"].filename = utils.get_path(["estimator_example", "supply_air_temperature.csv"])
model.components["020B_temperature_heating_setpoint"].filename = utils.get_path(["estimator_example", "temperature_heating_setpoint.csv"])
model.components["outdoor_environment"].filename_outdoorTemperature = utils.get_path(["estimator_example", "outdoor_environment.csv"])
model.components["outdoor_environment"].filename_globalIrradiation = utils.get_path(["estimator_example", "outdoor_environment.csv"])
model.components["outdoor_environment"].filename_outdoorCo2Concentration = utils.get_path(["estimator_example", "outdoor_environment.csv"])

## 3. Set Up Simulation Parameters

Define the simulation time period and step size. We'll simulate for several days to capture different operating conditions.

In [ ]:
# Set up simulation parameters
simulator = tb.Simulator(model)
step_size = 1200  # 20 minutes in seconds
start_time = datetime.datetime(year=2023, month=11, day=27, hour=0, minute=0, second=0,
                                tzinfo=tz.gettz("Europe/Copenhagen"))
end_time = datetime.datetime(year=2023, month=12, day=1, hour=0, minute=0, second=0,
                            tzinfo=tz.gettz("Europe/Copenhagen"))

print(f"Simulation period: {start_time} to {end_time}")
print(f"Step size: {step_size} seconds ({step_size/60:.1f} minutes)")


## 4. Identify Model Components

Let's identify the key components in our model that we'll be working with for parameter estimation.

In [ ]:
# Get references to key components
space = model.components["020B"]
space_heater = model.components["020B_space_heater"]
heating_controller = model.components["020B_temperature_heating_controller"]
co2_controller = model.components["020B_co2_controller"]
space_heater_valve = model.components["020B_space_heater_valve"]
supply_damper = model.components["020B_room_supply_damper"]
exhaust_damper = model.components["020B_room_exhaust_damper"]

print("Key components identified:")
print(f"- Building space: {space.id}")
print(f"- Space heater: {space_heater.id}")
print(f"- Heating controller: {heating_controller.id}")
print(f"- CO2 controller: {co2_controller.id}")
print(f"- Space heater valve: {space_heater_valve.id}")
print(f"- Supply damper: {supply_damper.id}")
print(f"- Exhaust damper: {exhaust_damper.id}")

## 5. Define Target Parameters for Estimation

We'll define the parameters we want to estimate. These include:
- Thermal parameters (capacitances, resistances, radiation factors)
- Mass parameters (volume, CO2 generation, infiltration)
- Space heater parameters (thermal mass, conductance)
- Controller parameters (PID gains)
- Valve and damper parameters (flow rates, authority)

Each parameter has initial values, lower bounds, and upper bounds.

In [ ]:

# component, attr, x0, lb, ub, [format]
parameters = [
    # Thermal parameters
    (space, "thermal.C_air", 2e+6, 1e+6, 1e+7),                    # Thermal capacitance of indoor air [J/K]
    (space, "thermal.C_wall", 2e+6, 1e+6, 1e+7),                   # Thermal capacitance of exterior wall [J/K]
    (space, "thermal.C_boundary", 5e+5, 1e+4, 1e+6),               # Thermal capacitance of exterior wall [J/K]
    (space, "thermal.R_out", 0.05, 0.01, 1),                       # Thermal resistance between wall and outdoor [K/W]
    (space, "thermal.R_in", 0.05, 0.01, 1),                        # Thermal resistance between wall and indoor [K/W]
    (space, "thermal.R_boundary", 0.01, 0.0001, 1),                # Thermal resistance between wall and indoor [K/W]
    (space, "thermal.f_wall", 0.3, 0, 1),                          # Radiation factor for exterior wall
    (space, "thermal.f_air", 0.1, 0, 1),                           # Radiation factor for air
    (space, "thermal.Q_occ_gain", 100.0, 10, 200),                 # Heat gain per occupant [W]
    
    # Mass parameters
    (space, "mass.V", 100, 10, 1000),                              # Volume of the space [m³]
    (space, "mass.G_occ", 8.18e-6, 1e-8, 1e-4),                   # CO2 generation rate per occupant [ppm·kg/s]
    (space, "mass.m_inf", 0.001, 1e-6, 0.3),                      # Infiltration rate [kg/s]
    
    # Space heater parameters
    (space_heater, "thermalMassHeatCapacity", 10000, 1000, 50000), # Thermal mass heat capacity [J/K]
    (space_heater, "UA", 30, 1, 100),                              # Thermal conductance [W/K]
    
    # Heating controller parameters (private - each controller gets its own values)
    (heating_controller, "kp", 0.001, 1e-5, 1, "private"),         # Proportional gain for heating controller
    (co2_controller, "kp", -0.001, -1, -1e-5, "private"),          # Proportional gain for CO2 controller
    ([heating_controller, co2_controller], "Ti", 10, 1, 100, "private"),             # Integral gain for heating controller
    ([heating_controller, co2_controller], "Td", 0, 0, 1, "private"),                # Derivative gain for controllers
    
    # Space heater valve parameters
    (space_heater_valve, "waterFlowRateMax", 0.01, 1e-6, 0.1),     # Maximum water flow rate [m³/s]
    (space_heater_valve, "valveAuthority", 0.8, 0.4, 1),           # Valve authority
    
    # Damper parameters (private - each damper gets its own values)
    ([supply_damper, exhaust_damper], "a", 1, 1, 10, "private"),                     # Shape parameter for supply damper
    ([supply_damper, exhaust_damper], "nominalAirFlowRate", 0.001, 1e-5, 1, "private"),  # Maximum air flow rate for supply damper [m³/s]
]


## 6. Configure Measuring Devices

Define the sensors that will provide measurement data for calibration. These sensors measure:
- Valve position
- Indoor temperature
- CO2 concentration
- Damper position

In [ ]:
percentile = 2
measurements = [
    (model.components["020B_valve_position_sensor"], 0.05/percentile),
    (model.components["020B_temperature_sensor"], 0.1/percentile),
    (model.components["020B_co2_sensor"], 30/percentile),
    (model.components["020B_damper_position_sensor"], 0.05/percentile)
]

print("Measuring devices for calibration:")
for device, sd in measurements:
    print(f"- {device.id}")

## 7. Run Initial Simulation

Before parameter estimation, let's run a simulation with the initial parameter values to see how well the model performs.

In [ ]:
# Run initial simulation for comparison
simulator.simulate(
    step_size=step_size,
    start_time=start_time,
    end_time=end_time
)

print("Initial simulation completed successfully!")

## 8. Plot Initial Results

Let's visualize the initial simulation results to understand the model behavior before calibration.

In [ ]:
# Plot initial results
fig, axes = tb.plot.plot_component(
    simulator,
    components_1axis=[
        ("020B", "indoorTemperature", "output"),
        ("outdoor_environment", "outdoorTemperature", "output"),
        (heating_controller.id, "setpointValue", "input"),
    ],
    components_2axis=[
        ("020B_space_heater", "Power", "output"),
        ("020B", "heatGain", "input"),
    ],
    components_3axis=[
        (heating_controller.id, "inputSignal", "output"),
    ],
    ylabel_1axis="Temperature [°C]",
    ylabel_2axis="Power [W]",
    ylabel_3axis="Water flow rate [m³/s]",
    title="Before calibration",
    show=False,
    nticks=11
)

print("Initial simulation plots generated.")

## 9. Create and Configure Estimator

Now we'll create an estimator instance and configure it for parameter estimation using automatic differentiation.

In [ ]:
# Create estimator
estimator = tb.Estimator(simulator)

# Configure optimization options
# These options are passed to the "method" we specify when we call estimator.estimate()
options = {} # Just use standard options

print("Estimator created and configured.")
print(f"Optimization options: {options}")

## 10. Run Parameter Estimation

Now we'll run the parameter estimation using the scipy solver SLSQP with automatic differentiation.

In [ ]:
# Run parameter estimation
# On a normal laptop cpu, this takes around 5 minutes
estimator.estimate(
        start_time,
        end_time,
        step_size,
        parameters,
        measurements,
        n_warmup=20,
        method=("scipy", "SLSQP", "ad"),
        n_cores=4,
        options=options,
    )



## 11. Plot Calibrated Results

Let's visualize the results after parameter calibration to see the improvement in model performance.

In [ ]:
# Plot results after calibration
fig, axes = tb.plot.plot_component(
    simulator,
    components_1axis=[
        ("020B", "indoorTemperature", "output"),
        (heating_controller.id, "setpointValue", "input"),
        (estimator.actual_readings[model.components["020B_temperature_sensor"].id], "Actual temperature"),
    ],
    components_2axis=[
        ("020B_space_heater", "Power", "output"),
    ],
    components_3axis=[
        ("020B_space_heater_valve", "valvePosition", "output"),
    ],
    ylabel_1axis="Temperature [°C]",
    ylabel_2axis="Power [W]",
    ylabel_3axis="Valve position [0-1]",
    title="After calibration",
    show=False,
    nticks=11
)

print("Calibrated simulation plots generated.")

## 12. Compare Valve Position

Let's specifically compare the valve position measurements to see how well the model matches the actual data.

In [ ]:
# Plot valve position comparison
fig, axes = tb.plot.plot_component(
    simulator,
    components_1axis=[
        ("020B_valve_position_sensor", "measuredValue", "input"),
        (estimator.actual_readings[model.components["020B_valve_position_sensor"].id], "Actual valve position"),
    ],
    ylabel_1axis="Valve position [0-1]",
    title="Valve position comparison",
    show=False,
    nticks=11
)

print("Valve position comparison plot generated.")

## 13. Compare Temperature Measurements

Finally, let's compare the temperature measurements to see the improvement in model accuracy.

In [ ]:
# Plot temperature comparison
fig, axes = tb.plot.plot_component(
    simulator,
    components_1axis=[
        ("020B_temperature_sensor", "measuredValue", "input"),
        (estimator.actual_readings[model.components["020B_temperature_sensor"].id], "Actual temperature"),
    ],
    ylabel_1axis="Temperature [°C]",
    title="Temperature comparison",
    show=True,
    nticks=11
)

print("Temperature comparison plot generated.")
print("\nParameter estimation example completed successfully!")

## Summary

In this example, we demonstrated how to:

1. **Load a pre-built model** from a semantic file containing building components
2. **Set up simulation parameters** including time period and step size
3. **Define target parameters** for estimation with appropriate bounds
4. **Configure measuring devices** to provide calibration data
5. **Run parameter estimation** using automatic differentiation
6. **Visualize results** before and after calibration

The parameter estimation process helps improve the accuracy of building models by calibrating them against real measurement data. This is essential for creating reliable digital twins that can be used for building control optimization, energy analysis, and predictive maintenance.

Key benefits of this approach:
- **Automatic differentiation** provides efficient gradient computation
- **Multiple parameter types** can be estimated simultaneously
- **Bounded optimization** ensures physically realistic parameter values
- **Visual comparison** helps validate the calibration results